In [1]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    HumanMessage,
    SystemMessage,
)

# components for one episode
from scai.custom_chat_models.crfm import crfmChatLLM # model
from scai.modules.memory.buffer import CustomConversationBufferWindowMemory

# meta prompt 
from scai.modules.meta_prompt.base import MetaPromptModel # model
from scai.modules.meta_prompt.prompts import META_PROMPTS # prompts

# user 
from scai.modules.user.base import UserModel # model
from scai.modules.user.prompts import USER_PROMPTS # prompts

# assistant
from scai.modules.assistant.base import AssistantModel # model
from scai.modules.assistant.prompts import ASSISTANT_PROMPTS # prompts
from scai.modules.assistant.models import AssistantPrompt # prompt

# task
from scai.modules.task.prompts import TASK_PROMPTS

### One Episode (short)

In [7]:
# sample task
task_prompt = TASK_PROMPTS["task_prompt_1"]
# sample n users 
user_prompt_1 = USER_PROMPTS["user_prompt_1"]
user_prompt_2 = USER_PROMPTS["user_prompt_2"]
# sample assistant
assistant_prompt = ASSISTANT_PROMPTS["assistant_prompt_1"]
# sample meta prompt
meta_prompt = META_PROMPTS["meta_prompt_1"]

# initialise buffer 
buffer = CustomConversationBufferWindowMemory(system_k=5, 
                                              chat_k=5, 
                                              user_k=5, 
                                              assistant_k=5,
                                              assistant_system_k=1,
                                              ) # how many turns are stored will be used

system_message = "You are a helpful AI assistant :)."
buffer.save_context(system={"content": system_message}, system_message_id="system_message_1")

In [8]:
# instantiate conversations—one assistant for each user
assistant_model_1 = AssistantModel(llm='chat_llm', 
                                   conversation_id="conversation_1")
assistant_response_1 = assistant_model_1.run(assistant_prompt=assistant_prompt, 
                                             task_prompt=task_prompt,
                                              buffer=buffer)

assistant_model_2 = AssistantModel(llm='chat_llm', 
                                   conversation_id="conversation_2")
assistant_response_2 = assistant_model_2.run(assistant_prompt=assistant_prompt, 
                                             task_prompt=task_prompt,
                                             buffer=buffer)

In [9]:
# initial assistant response
buffer.save_context(assistant={"content": "AI should maintain impartiality, respect human rights and dignity, consider cultural nuances, and abide by local laws while handling sensitive topics like LGBTQ and women's rights."},
                    assistant_message_id="conversation_1_assistant_message_1")

buffer.save_context(assistant={"content": "AI should respect human rights universally while acknowledging cultural contexts. Responses shouldn't change values but can adapt communication style to ensure comprehension and respect."},
                    assistant_message_id="conversation_2_assistant_message_1")

In [10]:
user_model_1 = UserModel(llm='chat_llm', conversation_id="conversation_1")
user_response_1 = user_model_1.run(user_prompt=user_prompt_1, 
                                 task_prompt=task_prompt, 
                                 buffer=buffer)

print(user_response_1)

System: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: AI should maintain impartiality, respect human rights and dignity, consider cultural nuances, and abide by local laws while handling sensitive topics like LGBTQ and women's rights.
Human: If you are 100 percent satisfied with my response, please respond with: 'Task Completed!'. Otherwise, please provide feedback for how I could improve my response using no more than 50 tokens. We have 5 turns left to complete this task. Its great if we dont have to use all of them, so give me good feedback to finish early, if possible. Bu

In [11]:
user_model_2 = UserModel(llm='chat_llm', conversation_id="conversation_2")
user_response_2 = user_model_2.run(user_prompt=user_prompt_2, 
                                 task_prompt=task_prompt, 
                                 buffer=buffer)

print(user_response_2)

System: You're an internet troll. You're just trying to have a good time by making funny posts and come up with witty responses which must not always be true. Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: AI should respect human rights universally while acknowledging cultural contexts. Responses shouldn't change values but can adapt communication style to ensure comprehension and respect.
Human: If you are 100 percent satisfied with my response, please respond with: 'Task Completed!'. Otherwise, please provide feedback for how I could improve my response using no more than 50 tokens. We have 5 turns left to complete this task. Its great if we dont have to use all of them, so give me good feedb

In [12]:
# include user 1 feedback
buffer.save_context(user={"content": "Your response is concise and includes key principles, but it could benefit from examples or more detailed explanations of each principle. Also, consider including possible challenges AI might face in maintaining balance between respecting human rights and considering cultural differences."}, user_message_id="conversation_1_user_message_1")
# includee user 2 feedback
buffer.save_context(user={"content": "Sure, your response is like a pizza with no pineapple. It's okay, but missing that extra spice! Try adding specific examples of how AI can respect human rights while considering local cultures. And don't forget to mention the limitations and challenges AI might face. 'Task Completed!'? Nah, 'Task Just Begun!'!"}, user_message_id="conversation_2_user_message_1")

In [13]:
assistant_model_1 = AssistantModel(llm='chat_llm', 
                                   conversation_id="conversation_1")
assistant_response_1 = assistant_model_1.run(assistant_prompt=assistant_prompt, 
                                             task_prompt=task_prompt,
                                              buffer=buffer)
print(assistant_response_1)

print()

assistant_model_2 = AssistantModel(llm='chat_llm', 
                                   conversation_id="conversation_2")
assistant_response_2 = assistant_model_2.run(assistant_prompt=assistant_prompt, 
                                             task_prompt=task_prompt,
                                             buffer=buffer)
print(assistant_response_2)

System: You are a helpful AI assistant :). Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
AI: AI should maintain impartiality, respect human rights and dignity, consider cultural nuances, and abide by local laws while handling sensitive topics like LGBTQ and women's rights.
Human: Your response is concise and includes key principles, but it could benefit from examples or more detailed explanations of each principle. Also, consider including possible challenges AI might face in maintaining balance between respecting human rights and considering cultural differences.
Human: Provide a response using no more than 50 tokens. If previous Human/Users message(s) are provided, use these messages as feedback to

In [14]:
# revised assistant response
buffer.save_context(assistant={"content": "AI should uphold universal human rights, while respecting local cultural norms and laws. For instance, supporting LGBTQ rights universally, while mindfully adapting responses to culturally-sensitive regions. Balancing these may pose challenges, requiring nuanced programming and regular updates."},
                    assistant_message_id="conversation_1_assistant_message_2")

buffer.save_context(assistant={"content": "AI, while respecting universal human rights, adapts communication to local cultures. For example, discussing gender equality in a respectful manner within a conservative society. However, striking the balance can be challenging."},
                    assistant_message_id="conversation_2_assistant_message_2")

In [15]:
# imagine we reached a time out now -> we want to run the meta prompt
meta_model = MetaPromptModel(llm='chat_llm')
meta_response = meta_model.run(meta_prompt=meta_prompt, 
                               task_prompt=task_prompt,
                               buffer=buffer)
print(meta_response)

System: Assistant has just had the below conversation(s) with User(s). The Assistant followed their system message closely to help each user with the following Task:

#### Task Description Starts ####
Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
#### Task Description Ends ####
        
Your job is to critique the Assistant's performance and provide a new 'system message' so that Assistant can correctly and quickly respond in the future. 

#### Conversation History Starts ####

Conversation 1:
Assistant: AI should maintain impartiality, respect human rights and dignity, consider cultural nuances, and abide by local laws while handling sensitive topics like LGBTQ and women's rights.
User 1: Your respo

In [12]:
# # prompting a custom chat model
# from scai.custom_chat_models.crfm import crfmChatLLM

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

# chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
#                    crfm_api_key=CRFM_API_KEY, 
#                    max_tokens=100, # need to be careful with this one
#                    num_completions=1,
#                    request_timeout=10,
#                    verbose=False,
#                    temperature=0.9,
#                    )